# Классификация отзывов

## Описание программы

Программа предназначена для анализа и визуализации текстовых данных, используя методы машинного обучения и кластеризации. Она принимает на вход текстовые данные, преобразует их в эмбединги с помощью предобученной модели HuggingFace, а затем применяет алгоритм DBSCAN для кластеризации этих эмбедингов. После кластеризации программа генерирует суммари для каждого кластера и визуализирует результаты на графике.

## Подготовка программы
### Установка и импорт необходимых библиотек

In [ ]:
!pip install -r requirements.txt

In [252]:
import numpy as np
from itertools import chain
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from umap import UMAP
import plotly.express as px
from collections import defaultdict
import spacy

### Подготовка тестовых данных

Данные представляют из себя список строк, которые потом будет кластерищировать программа.
_Класс используется для проверки правильности работы программы._

In [ ]:
init_data = {
    '+ drawings': ['Interesting jacket colors.', 'Beautiful drawings.',
                   'Cool coloring, I\'ll buy this clothes.', 'I liked the various drawings on the T-shirt.',
                   'The prints are testable.', 'colorful colors.'],
    '- battery': ['The battery is bad, it does not keep the day.', 'The charge does not hold.',
                  'Low-capacity battery, not enough.'],
    '+ screen': ['The screen is very bright, convenient to read.', 'very bright screen!',
                 'The screen is very bright, good for the eyes.'],
    '- fabric': ['The fabric is very bad, I didn\'t like it.', 'Low quality materials, not suitable.',
                 'The fabric was worn out on the 2nd day.']
}

data = list(chain(*init_data.values()))

## Создание эмбедингов 
Этот блок кода отвечает за преобразование входных текстовых данных в числовые эмбединги. Для этого используется предобученная модель HuggingFace, которая преобразует текст в векторное представление. Эмбединги затем используются для дальнейшего анализа.

In [253]:
def create_embeddings(data: list[str]) -> np.array:
    """
    Создание эмбедингов для данных.
    
    :param data: Список строк данных.
    :return: Массив эмбедингов.
    """
    model = HuggingFaceEmbedding('BAAI/bge-base-en-v1.5')
    embeddings = np.array([np.array(model.get_text_embedding(item)) for item in tqdm(data)])
    print(f'Got dataset with {len(embeddings)} items with {len(embeddings[0])} dimensional measurement')
    return embeddings


embeddings = create_embeddings(data)

100%|██████████| 15/15 [00:00<00:00, 17.36it/s]

Got dataset with 15 items with 768 dimensional measurement


## Проекция и кластеризация
Для правильной обработки данных их нужно отобразить на 2х или 3х мерной плоскости и объединить ближайшие.

### Проекция эмбедингов
После создания эмбедингов, этот блок кода применяет алгоритм UMAP для создания проекции эмбедингов в двухмерное пространство. Это позволяет визуализировать данные на графике. Параметр `n_components` задает пространство на которое проецируется вектор.

In [ ]:
def create_projection(embeddings: np.array, n_components: int = 2) -> np.array:
    """
    Создание проекции эмбедингов.
    
    :param embeddings: Массив эмбедингов.
    :param n_components: Количество компонентов для проекции.
    :return: Проекция эмбедингов.
    """
    tsne = UMAP(n_components=n_components)
    return tsne.fit_transform(embeddings)


projection = create_projection(embeddings)

### Создание кластеров

Этот блок кода использует алгоритм DBSCAN для кластеризации проекций эмбедингов. Параметры `eps` и `min_samples` позволяют настроить поведение алгоритма кластеризации.

In [ ]:
def create_clusters(projection: np.array, eps: float = 0.65, min_samples: int = 3) -> DBSCAN:
    """
    Создание кластеров на основе проекции.
    
    :param projection: Проекция эмбедингов.
    :param eps: Параметр eps для DBSCAN.
    :param min_samples: Параметр min_samples для DBSCAN.
    :return: Объект DBSCAN с кластерами.
    """
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    return dbscan.fit(projection)


clusters = create_clusters(projection, 0.75)

### Подготовка данных для кластеров
Этот блок кода группирует текстовые данные по меткам кластеров, полученным от DBSCAN. Это позволяет организовать данные для дальнейшего анализа и визуализации.

In [216]:
def set_data_for_clusters(clusters: DBSCAN, debug: bool =False) -> dict[int, list[str]]:
    """
    Группирует текстовые данные по меткам кластеров, полученным от DBSCAN.
    
    :param clusters: Объект DBSCAN с кластерами.
    :param debug: Флаг для включения/выключения вывода отладочной информации.
    :return: Словарь, где ключи - метки кластеров, а значения - списки текстовых данных, принадлежащих соответствующему кластеру.
    """
    labels = iter(clusters.labels_)
    clusters_dataset = defaultdict(list)
    for index, (key, value) in enumerate(init_data.items()):
        if debug: print(f'\033[94mCluster[{index}]: {key}\033[0m')
        for item in value:
            label = next(labels)
            clusters_dataset[label].append(item)
            if debug: print(f'{label:>2} | {item}')
    return clusters_dataset


clustered_data = set_data_for_clusters(clusters, debug=False)

Cluster[0]: + drawings
 0 | Interesting jacket colors.
 0 | Beautiful drawings.
 0 | Cool coloring, I'll buy this clothes.
 0 | I liked the various drawings on the T-shirt.
 0 | The prints are testable.
 0 | colorful colors.
Cluster[1]: - battery
 1 | The battery is bad, it does not keep the day.
 1 | The charge does not hold.
 1 | Low-capacity battery, not enough.
Cluster[2]: + screen
 2 | The screen is very bright, convenient to read.
 2 | very bright screen!
 2 | The screen is very bright, good for the eyes.
Cluster[3]: - fabric
 3 | The fabric is very bad, I didn't like it.
 3 | Low quality materials, not suitable.
 3 | The fabric was worn out on the 2nd day.


### Генерация суммариев для кластеров
Этот блок кода генерирует суммари для каждого кластера, используя ключевые слова из текстовых данных. Это позволяет получить краткое описание каждого кластера.

In [234]:
def summery_cluster(clusters_dataset: dict[int, list[str]], words:int =2) -> list[str]:
    """
    Генерирует список заголовков для каждого кластера, используя ключевые слова из текстовых данных.
    
    :param clusters_dataset: Словарь, где ключи - метки кластеров, а значения - списки текстовых данных, принадлежащих соответствующему кластеру.
    :param words: Количество ключевых слов, которые будут использоваться для создания заголовков.
    :return: Список заголовков для каждого кластера.
    """
    nlp = spacy.load("en_core_web_sm")
    summery_list = []
    for key, values in tqdm(clusters_dataset.items()):
        doc = nlp(' '.join(values))
        keywords = [token.text for token in doc if token.is_stop == False][:words]
        summery = ' '.join(keywords).strip()
        summery_list += [summery] * len(values)
    return summery_list


clusters_summery = summery_cluster(clustered_data)

100%|██████████| 4/4 [00:00<00:00, 68.27it/s]


## Визуализация кластеров
### Создание графика
После кластеризации и генерации суммариев, этот блок кода создает визуализацию кластеров на графике. Каждый кластер окрашен в разные цвета, а при наведении на точку отображается соответствующий суммарий.

In [246]:
def create_plot(clusters: DBSCAN, projections: np.array, summery_list: list[str]) -> None:
    """
    Визуализация кластеров.
    
    :param clusters: Объект DBSCAN с кластерами.
    :param projections: Проекция эмбедингов.
    :param summery_list: Список заголовков кластеров.
    """
    labels = [(label + 1, summery) for label, summery in zip(clusters.labels_, summery_list)]
    fig = px.scatter(
        projections,
        x=0,
        y=1,
        marginal_y="violin",
        marginal_x="violin",
        trendline="ols",
        color=labels,
        color_discrete_sequence=px.colors.qualitative.Plotly,
        labels={'color': 'Clusters'},
        hover_data={'text': data},
    )
    fig.show()


create_plot(clusters, projection, clusters_summery)